In [1]:
!pip install tesseract

Using legacy 'setup.py install' for tesseract, since package 'wheel' is not installed.
    Running setup.py install for tesseract: started
    Running setup.py install for tesseract: finished with status 'done'


You should consider upgrading via the 'C:\Users\oskar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
